# Searching for a Resonance 

### Omar Moreno (Santa Cruz Institute for Particle Physics, University of California, Santa Cruz)

## Introduction

If a heavy photon does indeed exist and has a mass that is within the acceptance of the Heavy Photon Search detector, it will appear as a resonance above the copious Quantum Electrodynamic trident invariant mass spectrum.  Such a signal would be Gaussian in shape, centered at the $A'$ mass with unknown normalization and a width equal to the mass resolution of HPS at the $A'$ mass. Searching for such a signal can be performed by constructing a window around the $A'$ mass hypothesis and fitting the selected range with a model composed of a polynomial, for the background, and a Gaussian for the signal.  The width and mean of the signal are taken as fixed parameters while the normalization of the Gaussian and coefficients of the polynomial are chosen such that they maximize the Poisson likelihood of the data.  The size of the window is dependent on the mass resolution.

In [ ]:
# This allows matplotlib plots to be shown inline
%matplotlib inline

import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import root_numpy as rnp
import ROOT as r

# Use the Bayesian Methods for Hackers style
plt.style.use('bmh')
plt.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size' : 20})

In [ ]:
# Path to the ROOT file that will be used to obtain the toy invariant mass distributions used to evaluate the fitter
file_path = "/home/omoreno/work/hps/analysis/engrun2015/pass4pt1/mc_1pt05/tridents/trident_analysis.root"

results_rec = rnp.root2array(file_path)

In [ ]:
bins = np.linspace(0, 0.1, 2000)

mass_arr = results_rec["invariant_mass"]
electron_chi2_arr = results_rec["electron_chi2"]
positron_chi2_arr = results_rec["positron_chi2"]
e_p_arr = results_rec["electron_p"]
p_p_arr = results_rec["positron_p"]
vx_arr = results_rec["vx"]
vy_arr = results_rec["vy"]
v0_p_arr = results_rec["v0_p"]

mass_arr_cuts = mass_arr[
            (electron_chi2_arr < 15) & (positron_chi2_arr < 15) &
            (e_p_arr < 0.85) & (p_p_arr < 0.85) &
            (((vx_arr*vx_arr/(0.04)) + (vy_arr*vy_arr/(0.0025))) < 1) &
            (v0_p_arr > 0.85)]

fig, ax0 = plt.subplots(nrows=1, ncols=1, figsize=(20, 10))

ax0.hist(mass_arr, bins, alpha=0.8, histtype="stepfilled", label="All")
ax0.hist(mass_arr_cuts, bins, alpha=0.8, histtype="stepfilled", label="Final Selection")
ax0.set_xlabel("Invariant Mass M($e^-e^+$) (GeV)")
ax0.set_ylabel("Events/0.000005")
ax0.set_yscale('log')
ax0.set_xlim(0, 0.08)
ax0.legend(loc=2);


## Mass Resolution

In [ ]:
mc_mr = np.genfromtxt("mc_mass_resolution.csv", 
                     dtype=[('mass', 'f8'), 
                            ('mass_error', 'f8'),
                            ('mass_sigma', 'f8'),
                            ('mass_sigma_error', 'f8')],
                     delimiter=","
                    )

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))

# Set the plot labels
ax0.set_xlabel("$m(ee)$ (MeV)")
ax0.set_ylabel("Mass Resolution (MeV)")

# Plot the mass resolutions as a function of mass
ax0.errorbar(mc_mr["mass"]*1000, mc_mr["mass_sigma"]*1000,
             xerr=mc_mr["mass_error"]*1000, yerr=mc_mr["mass_sigma_error"]*1000,
             fmt='o', markersize=10, label="$A'$ MC")
ax0.legend(loc=2)

results = np.polyfit(mc_mr["mass"]*1000,mc_mr["mass_sigma"]*1000, 3)
p = np.poly1d(results)
xp =  np.linspace(0, 100, 100)
ax0.plot(xp, p(xp), '-');
print results

residuals = mc_mr["mass_sigma"]*1000 - p(mc_mr["mass"]*1000)
ax1.errorbar(mc_mr["mass"]*1000, residuals, yerr=mc_mr["mass_sigma_error"]*1000, fmt='o');
ax1.set_xlabel("$m(e^{+}e^{-})$ (GeV)")
ax1.set_ylabel("Residual");

def get_mass_resolution(mass) : 
    return -6.166*math.pow(mass, 3) + 0.9069*math.pow(mass, 2) - 0.00297*mass + 0.000579

## Searching for a Resonance

As discussed in the introduction, if the heavy photon exist, it is expected to appear as a resonance (i.e. an excess) in the measured $e^+e^-$ invariant mass distribution.  The distribution of those events can be described by the probability density function (PDF) <br\>
<center>
$P(m_{e^+e^-}) =  \mu\phi(m_{e^+e^-} | m_{A'}, \sigma) + Bb(m_{e^+e^-}, a_i)$
</center>
Here, $\phi(m_{e^+e^-} | m_{A'}, \sigma)$ is a Gaussian with a mass set to the hypothesized $A'$ mass and width set to the measured experimental mass resolution, while the background, $b(m_{e^+e^-}, a_i)$, is a 7th order polynomial.

In [ ]:
# Independent variable
invariant_mass = r.RooRealVar("Invariant Mass", "Invariant Mass (GeV)", 0.0, 0.1)

#
# Signal PDF
#

ap_mass_hypothesis = 0.024525
print "A' mass hypothesis: %f" % (ap_mass_hypothesis)
# The mean of the signal must be set and can't vary
ap_mass_mean = r.RooRealVar("m_{A'}", "m_{A'}", ap_mass_hypothesis)

# The resolution of the signal is set to the value found experimentally
print "A' mass resolution: %f" % (get_mass_resolution(ap_mass_hypothesis))
ap_mass_sigma = r.RooRealVar("m_{\sigma}", "m_{\sigma}", get_mass_resolution(ap_mass_hypothesis))

# Create a gaussian signal pdf with the mean at the mass hypothesis and sigma
# set to the mass resolution
signal = r.RooGaussian("signal", "signal", invariant_mass, ap_mass_mean, ap_mass_sigma)

#
# Background PDF's
#

# Variables used to define polynomials
arg_list = r.RooArgList()
a = []
for order in xrange(1, 8) : 
    name = "t" + str(order)
    a.append(r.RooRealVar(name, name, -2, 2))
    arg_list.add(a[order - 1])
    
bkg = r.RooChebychev("bkg", 
                     "bkg", 
                     invariant_mass, 
                     arg_list)

# Number of events
nsig = r.RooRealVar("signal yield","signal yield", 0., -100000, 100000)
nbkg = r.RooRealVar("bkg yield","bkg yield", 300000., 100., 10000000.) 

#
# Composite model
#

comp_model = r.RooAddPdf("comp model", 
                         "comp model", 
                          r.RooArgList(signal, bkg), 
                          r.RooArgList(nsig, nbkg))

## Searching for a new particle

Establishing whether the signal+background model is significantly different from the background only model is typically done using the profile likelihood ratio and the test statistic $q_0$.